In [1]:
#import lightgbm
import random
import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
import time
from sklearn.metrics import make_scorer
t = time.strftime('%m%d-%H%M', time.localtime(time.time()))
import warnings
warnings.filterwarnings(action='ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정
#### alpha를 argument로 받는 함수로 실제 objective function을 wrapping하여 alpha값을 쉽게 조정할 수 있도록 작성했습니다.
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

 #점수 측정을 위한 코드
def SMAPE(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

깊이 튜닝

In [2]:
hyperparameters = pd.DataFrame(columns = ['n_estimators', 'eta', 'min_child_weight','max_depth', 'colsample_bytree', 'subsample'])


grid = {'n_estimators' : [100], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1),
        'max_depth' : np.arange(3,9,1) , 'colsample_bytree' :[0.8, 0.9],
        'subsample' : [0.8, 0.9]} # fix the n_estimators & eta(learning rate)

#tscv = TimeSeriesSplit(n_splits=3)
smape_score = make_scorer(SMAPE, greater_is_better=False)

for i in tqdm(np.arange(1, 101, 1)):
    train_df = pd.read_csv(f'./submit/train_building{i}.csv')
    # pretest_df = pd.read_csv(f'./pretest/pretest_building{i}.csv')
    ################################################
    #merge = pd.concat([train_df,pretest_df])
    ################################################
    #하이퍼 파라미터 튜닝 method 1
    #leaf method
    print(f"__________Buildding Number:{i} Hyperparameter Tuning__________")

    # preds = np.array([])
    grid_under = {}
    #합병된 데이터 사용
    y = train_df['power_consumption']
    x = train_df.drop(columns=['power_consumption'])
    #y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)
    #fit
    gcv = GridSearchCV(estimator = XGBRegressor(seed=0,tree_method="hist", gpu_id=0), param_grid=grid, scoring=smape_score, cv=PredefinedSplit(np.append(-np.ones(len(x)-168), np.zeros(168))) ,refit=True, verbose=False)
    gcv.fit(x, y)

    best = gcv.best_estimator_
    params = gcv.best_params_ 

    #pred
    # x_pretest = pretest_df.drop(columns=['power_consumption'])
    # y_pretest = pretest_df['power_consumption']
    # pred = best.predict(x_pretest)
    print("_____Best Parameters_____",params)
    # print("_____SMAPE Score________", SMAPE(y_pretest, pred))
    hyperparameters = pd.concat([hyperparameters,pd.DataFrame(params,index=[i-1])],axis=0)
hyperparameters.to_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv', index=False) # save the tuned parameters


  0%|          | 0/100 [00:00<?, ?it/s]

__________Buildding Number:1 Hyperparameter Tuning__________


  1%|          | 1/100 [00:10<17:30, 10.61s/it]

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 7, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:2 Hyperparameter Tuning__________


  2%|▏         | 2/100 [00:22<18:51, 11.55s/it]

...

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:99 Hyperparameter Tuning__________


 99%|█████████▉| 99/100 [18:35<00:12, 12.95s/it]

_____Best Parameters_____ {'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:100 Hyperparameter Tuning__________


100%|██████████| 100/100 [18:48<00:00, 11.29s/it]

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 7, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.8}


n_estimators 튜닝

In [3]:
#n_estimators
xgb_params = pd.read_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv')
scores = []   # smape 값을 저장할 list
best_it = []  # best interation을 저장할 list
for i in tqdm(range(100)):
    train_df = pd.read_csv(f'./submit/train_building{i+1}.csv')
    # pretest_df = pd.read_csv(f'./pretest/pretest_building{i+1}.csv')
    ################################################
    t_y = train_df['power_consumption']
    t_x = train_df.drop(['power_consumption'], axis=1)
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = t_y, X = t_x, test_size = 168)
    
    # p_y = pretest_df['power_consumption']
    # p_x = pretest_df.drop(['power_consumption'], axis=1)

    xgb_reg = XGBRegressor(n_estimators = 10000, tree_method="hist",  gpu_id=0, 
                           eta = 0.01, 
                           min_child_weight = xgb_params.iloc[i, 2],
                           max_depth = xgb_params.iloc[i, 3], 
                           colsample_bytree = xgb_params.iloc[i, 4],
                           subsample = xgb_params.iloc[i, 5], 
                           seed=0)
    xgb_reg.set_params(**{'objective':weighted_mse(100)}) # alpha = 100으로 고정

    xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_valid, y_valid)], early_stopping_rounds=300, verbose=False)
    
    # y_pred = xgb_reg.predict(p_x)
    # pred = pd.Series(y_pred)
    
    # sm = SMAPE(p_y, y_pred)
    # scores.append(sm)
    best_it.append(xgb_reg.best_iteration+1) ## 실제 best iteration은 이 값에 +1 해주어야 함.
hyperparameters['best_it'] = best_it

100%|██████████| 100/100 [02:44<00:00,  1.64s/it]


alpha튜닝

In [4]:
xgb_params = pd.read_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv')
#alpha
alpha_list = []
smape_list = []
for i in tqdm(range(100)):
    train_df = pd.read_csv(f'./pretest/train_building{i+1}.csv')
    pretest_df = pd.read_csv(f'./pretest/pretest_building{i+1}.csv')
    
    ################################################
    t_y = train_df['power_consumption']
    t_x = train_df.drop(['power_consumption'], axis=1)
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = t_y, X = t_x, test_size = 168)
    
    
    p_y = pretest_df['power_consumption']
    p_x = pretest_df.drop(['power_consumption'], axis=1)

    xgb = XGBRegressor(seed = 0, tree_method="hist",  gpu_id=0,
                      n_estimators = best_it[i], 
                      eta = 0.01, 
                      min_child_weight = xgb_params.iloc[i, 2],
                      max_depth = xgb_params.iloc[i, 3], 
                      colsample_bytree = xgb_params.iloc[i, 4], 
                      subsample = xgb_params.iloc[i, 5])

    xgb.fit(x_train, y_train ,verbose=False)
    pred0 = xgb.predict(x_valid)
    best_alpha = 0
    score0 = SMAPE(y_valid,pred0)

    for j in [1, 3, 5, 7, 10, 25, 50, 75, 100]:
        xgb = XGBRegressor(seed = 0,
                      n_estimators =  best_it[i], eta = 0.01, min_child_weight = xgb_params.iloc[i, 2],
                      max_depth = xgb_params.iloc[i, 3], colsample_bytree = xgb_params.iloc[i, 4], subsample = xgb_params.iloc[i, 5])
        xgb.set_params(**{'objective' : weighted_mse(j)})

        xgb.fit(x_train, y_train,verbose=False)
        
        pred1 = xgb.predict(x_valid)
        score1 = SMAPE(y_valid, pred1)
        if score1 < score0: 
            best_alpha = j
            score0 = score1

    alpha_list.append(best_alpha)
    smape_list.append(score0)
    
    print("building {} || best score : {} || alpha : {}".format(i+1, score0, best_alpha))
    
    
hyperparameters['alpha'] = alpha_list

  1%|          | 1/100 [00:06<10:54,  6.61s/it]

building 1 || best score : 5.559262975414478 || alpha : 1


  2%|▏         | 2/100 [00:11<09:02,  5.53s/it]

building 2 || best score : 8.40192043275441 || alpha : 75


  3%|▎         | 3/100 [00:13<06:24,  3.97s/it]

...

building 98 || best score : 9.878821964398634 || alpha : 5


 99%|█████████▉| 99/100 [09:08<00:04,  4.64s/it]

building 99 || best score : 3.775123097308505 || alpha : 5


100%|██████████| 100/100 [09:16<00:00,  5.56s/it]

building 100 || best score : 4.25430541635133 || alpha : 0


In [5]:
hyperparameters.to_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv')

In [6]:
# #smape점수 산출용
# preds = np.array([])
# smapes = []
# missfit_building = []
# hyper = pd.read_csv(f'./parameters/hyperparameter_xgb_wsw0808-1540.csv').drop(columns=['Unnamed: 0'])
# for i in tqdm(range(100)):
#     train_df = pd.read_csv(f'./pretest/train_building{i+1}.csv').drop(columns=['Unnamed: 0'])
#     pretest_df = pd.read_csv(f'./pretest/pretest_building{i+1}.csv').drop(columns=['Unnamed: 0'])
#     pred_df = pd.DataFrame()   # pretest 예측값을 담을 data frame
    
#     feature_importances = []
#     for seed in [0,1,2,3,4,5]: # 각 시드별 예측
#         y_train = train_df['power_consumption']
#         x_train = train_df.drop(columns=['power_consumption'])
        
#         x_pretest = pretest_df.drop(columns=['power_consumption'])
#         y_pretest = pretest_df['power_consumption']
        
#         xgb = XGBRegressor(seed = seed, tree_method="hist", gpu_id=0,          
#                            n_estimators = hyper.iloc[i,6], 
#                            eta = hyper.iloc[i, 1],
#                            min_child_weight = hyper.iloc[i, 2], max_depth = hyper.iloc[i, 3],
#                            colsample_bytree=hyper.iloc[i, 4], subsample=hyper.iloc[i, 5])

#         if hyper.iloc[i, 7] != 0:  # 만약 alpha가 0이 아니면 weighted_mse 사용
#             xgb.set_params(**{'objective':weighted_mse(hyper.iloc[i,7])})

#         xgb.fit(x_train, y_train)
#         feature_importances.append(xgb.feature_importances_)
#         y_pred = xgb.predict(x_pretest)
#         pred_df.loc[:,seed] = y_pred   # 각 시드별 예측 담기
    
#     pred = pred_df.mean(axis=1)   # (i+1)번째 건물의 예측 =  (i+1)번째 건물의 각 시드별 예측 평균값
#     pred_np = pred.to_numpy()
#     y_pretest_np = y_pretest.to_numpy()
#     score = SMAPE(y_pretest_np,pred_np) 
#     preds = np.append(preds, pred)
#     print("SMAPE:",score)
#     average_feature_importances = np.mean(feature_importances, axis=0)
#     plt.subplots(figsize=(6, 3))
#     plt.barh(x_train.columns, average_feature_importances)
#     plt.xlabel('Features')
#     plt.xlim(0,1)
#     plt.ylabel('Importance')
#     plt.show()
#     smapes.append(score)
    
#     if score > 6:
#         missfit_building.append(i)

In [7]:
# print(np.mean(smapes))